<a href="https://colab.research.google.com/github/the-SQuAD-squad/IR/blob/electra/IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DA FARE: 
-Sistemare dataset in modo che le coppie domanda passaggio siano giuste
-Provare modello dense, senza trainare Bert

In [1]:
#@title Init { form-width: "25%" }
import os
import random
import math
import numpy as np
import tensorflow as tf
import json
import pandas as pd
import re
import string
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm
!pip install tokenizers

from tokenizers import BertWordPieceTokenizer

!pip install transformers
import transformers
from transformers import AutoTokenizer
pd.set_option('display.max_colwidth', -1)

# fix random seeds
seed_value = 42 #@param {type:"integer"}

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

tf.compat.v1.set_random_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# BERT params
max_seq_length = 512
# Huggingface bert and associated tokenizer
pretrained_model_str = "google/electra-small-discriminator"
bert_hf_layer = transformers.TFElectraModel.from_pretrained(
    pretrained_model_str, output_attentions=True)

#pretrained_model_str = "roberta-base"
#bert_hf_layer = transformers.TFRobertaModel.from_pretrained(
    #pretrained_model_str, output_attentions=True)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_str)

!nvidia-smi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-small-discriminator.
If your task is similar to the task th

Tue Feb 23 19:11:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    40W / 250W |    515MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title df creation { form-width: "25%" }

# the official dataset is identical to the provided one
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O training_set.json

with open("training_set.json", "r") as f:
    json_file = json.load(f)
data = json_file["data"]

rows = []
for document in data:
  for par in document['paragraphs']:
    for qas in par['qas']:
      rows.append({
        'id' : qas['id'],
        'title': document["title"],
        'passage': par['context'],
        'question' : qas['question'],
        'answer_idx' : (qas['answers'][0]['answer_start'], 
                    qas['answers'][0]['answer_start'] + len(qas['answers'][0]['text'])),
        'answer_text' : qas['answers'][0]['text']
      })

df_original = pd.DataFrame(rows)

--2021-02-23 19:11:46--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘training_set.json’

training_set.json   100%[===================>]  28.88M  55.2MB/s    in 0.5s    

2021-02-23 19:11:47 (55.2 MB/s) - ‘training_set.json’ saved [30288272/30288272]



In [3]:
#@title clean dataset { form-width: "25%" }

!gcloud config set project feisty-mechanic-221914
!gsutil cp gs://squad_squad/error_IDs.txt ./error_IDs.txt

with open("error_IDs.txt", "r") as f:
    unwanted_id = f.read()

unwanted_id = unwanted_id.split("\n")[:-1]
df_bert = df_original.set_index('id')
df_bert = df_bert.drop(unwanted_id)
df_bert.head()

Updated property [core/project].
Copying gs://squad_squad/error_IDs.txt...
/ [1 files][  5.7 KiB/  5.7 KiB]                                                
Operation completed over 1 objects/5.7 KiB.                                      


,title,passage,question,answer_idx,answer_text
id,,,,,
5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"(515, 541)",Saint Bernadette Soubirous
5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is in front of the Notre Dame Main Building?,"(188, 213)",a copper statue of Christ
5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",The Basilica of the Sacred heart at Notre Dame is beside to which structure?,"(279, 296)",the Main Building
5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is the Grotto at Notre Dame?,"(381, 420)",a Marian place of prayer and reflection
5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary rep

In [4]:

pairs_true = [[passage , question] for passage, question in zip(df_bert['passage'],df_bert['question'])]
labels_true = [1]*len(pairs_true)

pairs_false = []

groups = df_bert.groupby("title").groups

for index in tqdm(df_bert.index):
    title = df_bert["title"].loc[index]
    passage = df_bert["passage"].loc[index]
    group = groups[title]
    different_passages = set()
    for id in group:
        if df_bert["passage"][id] != passage:
            different_passages.add(df_bert["passage"][id])
    for passage in list(different_passages)[:1]: #take one for bilanced dataset
        pairs_false.append([passage,df_bert['question'][index]])

labels_false = [0]*len(pairs_false)


print(len(pairs_true),len(pairs_false),sep='\n')


87364
87364


In [23]:
df_ir = pd.DataFrame({
'pairs' : pairs_true + pairs_false,
'labels': labels_true + labels_false
})
df_ir.head()

,pairs,labels
0,"[Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary., To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?]",1
1,"[Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary., What is in front of the Notre Dame Main Building?]",1
2,"[Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary., The Basilica of the Sacred heart at Notre Dame is beside to which structure?]",1
3,"[Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary., What is the Grotto at Notre Dame?]",1
4,"[Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary., What sits on top of the Main Building at Notre Dame?]",1


In [26]:
a = []
b = []
c = []
for passage,question in tqdm(df_ir['pairs']):
    out = tokenizer(passage,question, padding="max_length", max_length=max_seq_length, truncation=True,verbose=True)
    a.append(out['input_ids'])
    b.append(out['attention_mask'])
    c.append(out['token_type_ids'])

a = np.stack(a)
b = np.stack(b)
c = np.stack(c)
x_train = [a,b,c]
y_train = np.array(df_ir['labels'])


In [41]:
#@title model definition { form-width: "25%" }
import tensorflow as tf
from tensorflow.keras.losses import cosine_similarity, MSE, mae

bert_hf_layer = transformers.TFElectraModel.from_pretrained(pretrained_model_str)

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')

encoding = bert_hf_layer(input_ids=input_word_ids, attention_mask=input_mask, 
                                token_type_ids=input_type_ids).last_hidden_state[:,0]

output = tf.keras.layers.Dense(1,activation= "sigmoid")(encoding)

model = keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], 
                    outputs=output,
                    name="BERT_IR")

optimizer = keras.optimizers.Adam()

model.summary(line_length=150)

Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-small-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.


Model: "BERT_IR"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_word_ids (InputLayer)                      [(None, 512)]                    0                                                                   
______________________________________________________________________________________________________________________________________________________
input_mask (InputLayer)                          [(None, 512)]                    0                                                                   
______________________________________________________________________________________________________________________________________________________
input_type_ids (InputLayer)                      [(None, 512)]               

In [ ]:
#@title train { form-width: "25%" }

from transformers.optimization_tf import AdamWeightDecay
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

batch_size = 32
epochs = 200
steps_per_epoch = 20

saveDir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)
chkpt = saveDir + '/squad_check.hdf5'

es_cb = EarlyStopping(monitor='val_loss', patience=2,verbose=1, mode='auto')
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, 
                        save_best_only=False, mode='auto', 
                        save_weights_only=True)

callbacks = [es_cb, cp_cb]

ENABLE_WANDB = False        #@param {type:"boolean"}
wandb_experiment_name = "HF_" + bert_hf_layer.name  #@param {type: "string"}
if ENABLE_WANDB:
    !pip install wandb > /dev/null
    !wandb login
    import wandb
    from wandb.keras import WandbCallback
    wandb.init(project="SQUAD", name=wandb_experiment_name)
    wandb.config.batch_size = batch_size
    wandb.config.epochs = epochs
    callbacks.append(WandbCallback(log_batch_frequency=10,
                                   save_weights_only=True))

optimizer = AdamWeightDecay(lr=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics = "accuracy")
history = model.fit(x=x_train, y=y_train, epochs=epochs, batch_size=batch_size, 
                    validation_split=.1, callbacks=callbacks)

Epoch 1/200
 284/4915 [>.............................] - ETA: 35:41 - loss: 0.7306 - accuracy: 0.5135

In [20]:
len(x_train[0])

174728

In [ ]:
# CELLA DI PROVA INFERENCE
#@title inference model PROVA { form-width: "25%" }
from sklearn.metrics.pairwise import cosine_similarity

batch_size = 16

all_encodings = []

for idx in tqdm(range(len(df_bert_val)//batch_size)):
    questions = df_bert_val['question'].iloc[idx * batch_size : (idx + 1) * batch_size].tolist()
    attention_mask_questions = np.stack(questions)
    attention_mask_questions[attention_mask_questions != 0] = 1
    passages = df_bert_val['passage'].iloc[idx * batch_size : (idx + 1) * batch_size].tolist()
    attention_mask_passages = np.stack(passages)
    attention_mask_passages[attention_mask_passages != 0] = 1
    encodings = model.predict([np.stack(passages), attention_mask_passages, np.stack(questions), attention_mask_questions])
    all_encodings.append(encodings)
    break
    # encoded_val_passage[i*batch_size : (i+1)*batch_size] = inf_model(passages[i*batch_size : (i+1)*batch_size])
    # encoded_val_question[i*batch_size : (i+1)*batch_size] = inf_model(questions[i*batch_size : (i+1)*batch_size])

#print(encodings.shape)

# all_encodings_arr = np.vstack(all_encodings)
# encoding_passages = all_encodings_arr[:,:,0]
# encoding_questions = all_encodings_arr[:,:,1]

encoding_passages = encodings[:,:,0]
encoding_questions = encodings[:,:,1]
first_passage_encoding = encoding_passages[0]
first_question_encoding = encoding_questions[0]
#print(first_passage_encoding.reshape(1,-1).shape)
print(cosine_similarity(first_question_encoding.reshape(1,-1),first_passage_encoding.reshape(1,-1)))
#print(cosine_similarity(encoding_questions, encoding_passages))

In [ ]:
# SECONDA CELLA PROVA INFERENCE
cos_sim = cosine_similarity(encoding_questions, encoding_passages)
print(np.argmax(cos_sim[878]))
ok=0
for i in range(7136):   # i is the question index
    best = np.argmax(cos_sim[i])
    if np.array_equal(df_bert_val.iloc[i].passage, df_bert_val.iloc[best].passage):
        ok+=1

In [ ]:
len(df_bert_val)

In [ ]:
print(ok/7136)

In [ ]:
print(cos_sim[0][:100])

In [ ]:
df_bert_val.iloc[:100]

In [ ]:
# TERZA CELLA PROVA INFERENCE
df_bert_val.iloc[3034]
df_bert.loc["571a48c110f8ca1400304fc7"]
# When did Beyonce start becoming popular? 
# Who began producing Oswald cartoons for Universal in 1929?  
#What is another term for specific formal prescriptions?   

In [ ]:
#@title inference model { form-width: "25%" }

input_txt_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32)
output = bert_hf_layer(input_ids=input_txt_ids).last_hidden_state[:,0]
inf_model = keras.Model(inputs=input_txt_ids, outputs=output, name="BERT_IR")

encoded_val_passage = np.zeros((len(df_bert_val), 256))
encoded_val_question = np.zeros((len(df_bert_val), 256))
passages = np.stack(df_bert_val['passage'])
questions = np.stack(df_bert_val['question'])


for i in tqdm(range(len(df_bert_val)//batch_size)):
    encoded_val_passage[i*batch_size : (i+1)*batch_size] = inf_model(passages[i*batch_size : (i+1)*batch_size])
    encoded_val_question[i*batch_size : (i+1)*batch_size] = inf_model(questions[i*batch_size : (i+1)*batch_size])

In [ ]:
#@title kNN cosine similarity { form-width: "25%" }

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

k=1
tree = NearestNeighbors(n_neighbors=k, metric='cosine')
tree.fit(encoded_val_passage)

results = tree.kneighbors(encoded_val_question, n_neighbors=k, return_distance=False)

ok=0
for i in range(len(df_bert_val)):
    for j in range(k):
        if np.array_equal(passages[results[i,j]], passages[i]):
            ok+=1

ok/len(df_bert_val)*100

In [ ]:
results[0]

In [ ]:
!pip3 install wikipedia-api

import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')
cat = wiki_wiki.page("Category:Physics")
print("Category members: Category:Physics")
for p in cat.categorymembers.values():
  if p.namespace == wikipediaapi.Namespace.CATEGORY:
    # it is category, so you have to make decision
    # if you want to fetch also text from pages that belong
    # to this category
    print(p)
  elif p.namespace == wikipediaapi.Namespace.MAIN:
    # it is page => we can get text
    print(p)
    print(p.text)

def print_categorymembers(categorymembers, level=0, max_level=3):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
                print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)

cat = wiki_wiki.page("Category:Artificial intelligence")
print_categorymembers(cat.categorymembers)

wiki_wiki = wikipediaapi.Wikipedia('en')

page_py = wiki_wiki.page('Python_(programming_language)')
print(page_py.text)